# Welcome to AmpForm-DPD!

```{title} Welcome
```

[![Supported Python versions](https://img.shields.io/pypi/pyversions/ampform-dpd)](https://pypi.org/project/ampform-dpd)
[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ComPWA/ampform/blob/main)
[![Binder](https://static.mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ComPWA/ampform/main?urlpath=lab)

This Python package is a (temporary) extension of [AmpForm](https://ampform.rtfd.io) and provides a symbolic implementation of Dalitz-plot decomposition ([10.1103/PhysRevD.101.034033](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.101.034033)) with [SymPy](https://www.sympy.org/en/index.html). It has been extracted from the [ComPWA/polarimetry](https://github.com/ComPWA/polarimetry) repository, which is not yet public.

## Installation

The fastest way of installing this package is through PyPI ([`ampform-dpd`](https://pypi.org/project/ampform-dpd)):

```shell
python3 -m pip install ampform-dpd
```

This installs the latest version that you can find on the [`stable`](https://github.com/ComPWA/ampform-dpd/tree/stable) branch. The latest version on the [`main`](https://github.com/ComPWA/ampform/tree/main) branch
can be installed as follows:

```shell
python3 -m pip install git+https://github.com/ComPWA/ampform@main
```

You can substitute `main` in the above command with any of the [tags](https://github.com/ComPWA/ampform-dpd/tags) listed under the [releases](https://github.com/ComPWA/ampform-dpd/releases) or any other commit hashes. However, we highly recommend using the more dynamic, {ref}`'editable installation' <compwa:develop:Editable installation>` instead. This goes as follows:

1. Get the source code (see [the Pro Git Book](https://git-scm.com/book/en/v2)):

   ```shell
   git clone https://github.com/ComPWA/ampform-dpd
   cd ampform-dpd
   ```

2. **\[Recommended\]** Create a virtual environment (see {ref}`here <compwa:develop:Virtual environment>` or the tip below).

3. Install the project in {ref}`'editable installation' <compwa:develop:Editable installation>` with {ref}`additional dependencies <compwa:develop:Optional dependencies>` for the developer:

   ```shell
   python3 -m pip install -e . --group dev
   ```

That's all! Have a look at the {doc}`/index` page to try out the package, and see {doc}`compwa:develop` for tips on how to work with this 'editable' developer setup!

:::{tip}
It's easiest to install the project in a virtual environment with [`uv`](https://docs.astral.sh/uv). In that case, to install in editable mode, just run:

```shell
uv sync
source .venv/bin/activate
```

This way of installing is also safer, because it {ref}`pins all dependencies <compwa:develop:Pinning dependency versions>`.
:::

<!-- cspell:ignore pkpi -->

## Physics

Dalitz-plot decomposition allows us to separate variables that affect the <font color=RoyalBlue>angular distribution</font> from variables that describe the <font color=Orange>dynamics</font>. It allows rewriting a **transition amplitude**&nbsp;$T$ as

$$
T^{\Lambda}_{\{\lambda\}}(\alpha,\beta,\gamma; \{\sigma\}) = \sum_{\nu}
{\color{RoyalBlue} D_{\Lambda,\nu}^{J}(\alpha,\beta,\gamma)}
\,
{\color{Orange} O^\nu_{\{\lambda\}}(\{\sigma\})}.
$$

Here, $\Lambda$ and $\nu$ indicate the allowed spin projections of the initial state, $\{\lambda\}$ are the allowed spin projections of the final state (e.g. $\{\lambda\}=\lambda_1,\lambda_3,\lambda_3$ for a three-body decay). The Euler angles $\alpha,\beta,\gamma$ are obtained by choosing a specific aligned center-of-momentum frame ("aligned CM"), see Fig.&nbsp;2 in Ref&nbsp;{cite}`JPAC:2019ufm`, which gives us an "aligned" transition amplitude $O^\nu_{\{\lambda\}}$ that only depends on dynamic variables $\{\sigma\}$ (in the case of a three-body decay, the three Mandelstam variables $\sigma_1,\sigma_2,\sigma_3$).

These aligned transition amplitudes are then combined into an observable **differential cross section** (intensity distribution), using a spin density matrix $\rho_{_{\Lambda,\Lambda'}}$ for the spin projections $\Lambda$ of the initial state,

$$
\mathrm{d}\sigma/\mathrm{d}\Phi_3 = N
\sum_{\Lambda,\Lambda'} \rho_{_{\Lambda,\Lambda'}}
\sum_{\nu,\nu'} {\color{RoyalBlue}
      D^{J^*}_{\Lambda,\nu}\left(\alpha,\beta,\gamma\right)
      D^{J}_{\Lambda',\nu'}\left(\alpha,\beta,\gamma\right)
   }
   \sum_{\{\lambda\}} {\color{Orange}
      O^\nu_{\{\lambda\}}\left(\{\sigma\}\right)
      O^{\nu'*}_{\{\lambda\}}\left(\{\sigma\}\right)
   }.
$$

Given the right alignment, the aligned transition amplitude can be written as

```{math}
---
label: aligned-amplitude
---
\begin{eqnarray}
{\color{Orange}O^\nu_{\{\lambda\}}\left(\{\sigma\}\right)} &=&
   \sum_{(ij)k}
   \sum^{(ij)\to i,j}_s
   \sum_\tau
   \sum_{\{\lambda'\}}
   \; {\color{Orange}X_s\!\left(\sigma_k\right)}
\\ {\color{LightGray}\text{production:}} && \quad \times\;
   \eta_J\,
   d^J_{\nu,\tau-\lambda'_k}\!\left(\hat\theta_{k(1)}\right)\,
   H^{0\to(ij),k}_{\tau,\lambda_k'}\,(-1)^{j_k-\lambda_k'}
\\ {\color{LightGray}\text{decay:}} && \quad \times\;
   \eta_s\,
   d^s_{\tau,\lambda'_i-\lambda_j'}\!\left(\theta_{ij}\right)\,
   H^{(ij)\to i,j}_{\lambda'_i,\lambda'_j}\,(-1)^{j_j-\lambda_j'}
\\ {\color{LightGray}\text{rotations:}} && \quad \times\;
   d^{j_1}_{\lambda'_1,\lambda_1}\!\left(\zeta^1_{k(0)}\right)\,
   d^{j_2}_{\lambda'_2,\lambda_2}\!\left(\zeta^2_{k(0)}\right)\,
   d^{j_3}_{\lambda'_3,\lambda_3}\!\left(\zeta^3_{k(0)}\right).
\end{eqnarray}
```

Notice the general structure:

- **Summations**: The outer sum is taken over the three decay chain combinations $(ij)k \in \left\{(23)1, (31)2, (12)3\right\}$. Next, we sum over the spin magnitudes&nbsp;$s$ of all resonances[^1], the corresponding allowed helicities&nbsp;$\tau$, and allowed spin projections&nbsp;$\{\lambda'\}$ of the final state.
- **Dynamics**: The function $X_s$ only depends on a single Mandelstam variable and carries all the dynamical information about the decay chain. Typically, these are your $K$-matrix or Breit-Wigner lineshape functions.
- **Isobars**: There is a Wigner&nbsp;$d$-function and a helicity coupling $H$ for each isobar in the three-body decay chain: the $0\to(ij),k$ production node and the $(ij)\to i,j$ decay node. The argument of these Wigner&nbsp;$d$-functions are the polar angles. The factors $\eta_J=\sqrt{2S+1}$ and $\eta_s=\sqrt{2s+1}$ are normalization factors. The phase $(-1)^{j-\lambda}$ is added to both helicity couplings to convert to the Jacob-Wick particle-2 convention.[^2] The convention treats the first and the second particle unequally, however, it enables the simple relation of the helicity couplings to the $LS$&nbsp;couplings explained below.
- **Wigner rotations**: The last three Wigner&nbsp;$d$-functions represent Wigner rotations that appear when rotating the boosted frames of the production and decay isobar amplitudes back to the space-fixed CM frame.

If $k=1$, we have $\hat\theta_{k(1)}=0$, so the Wigner&nbsp;$d$ function for the production isobar reduces to a Kronecker delta, $d^J_{\nu,\tau-\lambda'_k}\!\left(\hat\theta_{k(1)}\right) = \delta_{\nu,\tau-\lambda'_k}$.

[^1]: Alternatively, one can sum over all resonances themselves, so that one has a dynamic function&nbsp;$X_\mathcal{R}(\sigma_k)$ for each resonance&nbsp;$\mathcal{R}$ in subsystem&nbsp;$k$.

[^2]: See also {cite}`JPAC:2021rxu`, Section&nbsp;2.1.

Equation&nbsp;{eq}`aligned-amplitude` is written in terms of _helicity_ couplings, but can be rewritten in terms of _$LS$&nbsp;couplings_, using

$$
\begin{aligned}
H^{0\to(ij),k}_{\tau,\lambda'_k} \;&=\;
   \sum_{LS}
   H^{0\to(ij),k}_{LS}
   \sqrt{\frac{2L+1}{2J+1}}
   C^{S,\tau-\lambda'_k}_{s,\tau,j_k,-\lambda'_k}
   C^{J,\tau-\lambda'_k}_{L,0,S,\tau-\lambda'_k} \\
H^{(ij)\to i,j}_{\lambda'_i,\lambda'_j} \;&=\;
   \sum_{l's'}
   H^{0\to(ij),k}_{l's'}
   \sqrt{\frac{2l'+1}{2s+1}}
   C^{s',\lambda'_i-\lambda'_j}_{j_i,\lambda'_i,j_j,-\lambda'_j}
   C^{s,\lambda'_i-\lambda'_j}_{l',0,s',\lambda'_i-\lambda'_j}\,.
\end{aligned}
$$

The dynamics function is dependent on the $LS$&nbsp;values and we write $X_s^{LS;l's'}$ instead of $X_s$.

## Usage

The core class of this package is the {class}`~ampform_dpd.DalitzPlotDecompositionBuilder`. It requires a {class}`.ThreeBodyDecay` object to formulate amplitude models. You can create such a decay object by hand, but it is easier to generate it with [QRules](https://qrules.rtfd.io). Here is an example for the decay $J/\psi \to p \bar{\varSigma}^- \bar{K}^0$. In the last step, we "normalize" the state IDs of the incoming and outgoing states so that they match the index naming scheme of the [Dalitz-Plot Decomposition paper](https://journals.aps.org/prd/abstract/10.1103/PhysRevD.101.034033).

In [ ]:
import qrules

from ampform_dpd.adapter.qrules import normalize_state_ids

reaction = qrules.generate_transitions(
    initial_state="J/psi(1S)",
    final_state=["p", "Sigma~-", "K~0"],
    allowed_interaction_types="strong",
    mass_conservation_factor=0,
)
reaction = normalize_state_ids(reaction)

In [ ]:
import graphviz

dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

Since AmpForm-DPD formulates amplitudes for three-body decays only, it defines its own topology class for encoding the decay chains. You can convert the QRules {class}`~qrules.transition.ReactionInfo` object to such a {class}`.ThreeBodyDecay` as follows:

In [ ]:
from ampform_dpd.adapter.qrules import to_three_body_decay

decay = to_three_body_decay(reaction.transitions)

The {mod}`ampform_dpd.io` module offers some handy functions for rendering the decay chains in different format.

In [ ]:
from IPython.display import Latex, Markdown

from ampform_dpd.io import as_markdown_table

Markdown(as_markdown_table(decay))

In [ ]:
from IPython.display import Math

from ampform_dpd.io import aslatex

Math(aslatex(decay))

In [ ]:
Math(aslatex(decay, with_jp=True))

In [ ]:
Markdown(as_markdown_table([decay.initial_state, *decay.final_state.values()]))

In [ ]:
resonances = sorted(
    {t.resonance for t in decay.chains},
    key=lambda p: (p.name[0], p.mass),
)
Markdown(as_markdown_table(resonances))

We an now use the decay object to formulate amplitude models. Notice that we do not parametrize the dynamics for each of the chains yet, we only have a simple expression for the unpolarized differential cross-section ("intensity") that contains the expected Wigner-$d$ functions for the spin alignment.

In [ ]:
from ampform_dpd import DalitzPlotDecompositionBuilder

model_builder = DalitzPlotDecompositionBuilder(decay)
no_dynamics_model = model_builder.formulate()
no_dynamics_model.intensity

Unfortunately, {class}`sympy.Indexed<sympy.tensor.indexed.Indexed>` symbols with a superscript don't render nicely. To fix this, AmpForm-DPD provides the following configuration function.

In [ ]:
from ampform_dpd.io import simplify_latex_rendering

simplify_latex_rendering()

In [ ]:
no_dynamics_model.intensity

More interesting is to formulate a model with a dynamics parametrization. The most common choice is a relativistic Breit–Wigner function. It can optionally have form factors for the production vertex and/or for the decay vertex and have an energy-dependent width.

In [ ]:
from ampform_dpd.dynamics.builder import BreitWignerBuilder

dynamics_builder = BreitWignerBuilder(
    energy_dependent_width=True,
    production_form_factor=True,
    decay_form_factor=True,
)
for chain in model_builder.decay.chains:
    model_builder.dynamics_choices.register_builder(chain, dynamics_builder)
model = model_builder.formulate()

...or everything as a one-liner:

In [ ]:
from ampform_dpd import DalitzPlotDecompositionBuilder
from ampform_dpd.dynamics.builder import BreitWignerBuilder

model_builder = DalitzPlotDecompositionBuilder(decay)
dynamics_builder = BreitWignerBuilder()
for chain in model_builder.decay.chains:
    model_builder.dynamics_choices.register_builder(chain, dynamics_builder)
model = model_builder.formulate()

In [ ]:
selection = {k: v for i, (k, v) in enumerate(model.amplitudes.items()) if i % 13 == 0}
Math(aslatex(selection, terms_per_line=1))

Here's an example with a simple Breit–Wigner without form factors:

In [ ]:
bw_builder = BreitWignerBuilder(
    energy_dependent_width=False,
    production_form_factor=False,
    decay_form_factor=False,
)
for chain in model_builder.decay.chains:
    model_builder.dynamics_choices.register_builder(chain, bw_builder)
simple_model = model_builder.formulate()

In [ ]:
(amp_symbol, amp_expr), *_ = simple_model.amplitudes.items()
Math(aslatex({amp_symbol: amp_expr}, terms_per_line=1))

Custom dynamics can be defined by defining a dynamics builder function that has a signature that matches the {class}`.DynamicsBuilder` protocol.

In [ ]:
import sympy as sp

from ampform_dpd import DefinedExpression
from ampform_dpd.decay import ThreeBodyDecayChain
from ampform_dpd.dynamics.builder import create_mass_symbol, get_mandelstam_s


def formulate_gaussian(decay_chain: ThreeBodyDecayChain) -> DefinedExpression:
    resonance = decay_chain.resonance
    s = get_mandelstam_s(decay_chain.decay_node)
    x = sp.sqrt(s)
    mu = create_mass_symbol(resonance)
    sigma2 = sp.Symbol(Rf"\sigma_{{{resonance.latex}}}^2", nonnegative=True)
    return DefinedExpression(
        expression=sp.exp(-((x - mu) ** 2) / (2 * sigma2)),
        parameters={
            mu: resonance.mass,
            sigma2: resonance.width**2,
        },
    )


for chain in model_builder.decay.chains:
    model_builder.dynamics_choices.register_builder(chain, formulate_gaussian)
gaussian_model = model_builder.formulate()

In [ ]:
(amp_symbol, amp_expr), *_ = gaussian_model.amplitudes.items()
Math(aslatex({amp_symbol: amp_expr}, terms_per_line=1))

The {meth}`.DalitzPlotDecompositionBuilder.formulate` method has other arguments that allow you to select a different reference subsystem and to use **one complex-valued coefficient** rather than two couplings per decay vertex. Also notice the tricks used here to hide the trivial summations over the helicities of spin-0 particles.

In [ ]:
for chain in model_builder.decay.chains:
    model_builder.dynamics_choices.register_builder(chain, bw_builder)
coefficient_model = model_builder.formulate(
    cleanup_summations=True,
    reference_subsystem=2,
    use_coefficients=True,
)
coefficient_model.intensity.cleanup()

In [ ]:
(symbol, expr), *_ = coefficient_model.amplitudes.items()
Latex(aslatex({symbol: expr}, terms_per_line=1))

## Examples

```{toctree}
---
maxdepth: 1
---
lc2pkpi
jpsi2ksp
xib2pkk
serialization
```

```{toctree}
---
maxdepth: 2
---
comparison
```

```{toctree}
---
hidden:
---
references
API <api/ampform_dpd>
```

```{toctree}
---
caption: Related projects
hidden:
---
QRules <https://qrules.readthedocs.io>
AmpForm <https://ampform.readthedocs.io>
TensorWaves <https://tensorwaves.readthedocs.io>
PWA Pages <https://pwa.readthedocs.io>
ComPWA project <https://compwa.github.io>
```